<a href="https://colab.research.google.com/github/jooeun921/Big-Data-Analyst/blob/main/Part05_statistical_estimation_hypothesis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 05. 통계적 추정과 검정
Python을 활용한 통계적 추정과 가설 검정. 작업형 제 3유형에서 나옴.
- 통계적 추정과 가설 검정의 기본 개념 : 점추정, 구간추정, p-value 해석 및 가설 검정 원리
- t검정 : 1표본, 독립 2 표본, 대응표본 t-검정
- 데이터 분포 확인과 정규성 검정 : 다섯 숫자 요약, Q-Q Plot, 샤피로-윌크 검정 및 카이제곱 적합도 검정
- 분산분석(ANOVA) : 그룹 간 평균 차이 검정, F 검정 및 Tukey의 HSD 사후 검정
- 비모수 검정(Non-Parametric Test) : 윌콕슨 손위합 검정, 맨-휘트니 U 검정, 부호 검정 및 Levene 검정을 통한 등분산 검정

### Section 01 학습 : 통계적 추정과 가설 검정

#### 구간추정

In [ ]:
# 표본평균을 사용한 모평균 추정

import numpy as np
import matplotlib.pyplot as plt

mu, sigma = 30, 5
sample_sizes = [10, 50, 100, 500, 1000, 5000]
num_means = 1000

fig, axes = plt.subplots(2, 3, figsize = (15, 15))
axes = axes.flatten()

x_limits = (mu - 3 * sigma, mu + 3 * sigma)
for i, size in enumerate(sample_sizes):
    sample_means = [np.mean(np.random.normal(mu, sigma, size)) for _ in range(num_means)]
    axes[i].hist(sample_means, bins = 30, alpha = 0.75, color = 'blue', edgecolor = 'black')
    axes[i].axvline(mu, color = 'green', linestyle = 'dashed', linewidth = 1)
    axes[i].set_xlim(x_limits)
    axes[i].set_title(f'Sample Size: {size}\nNumber of Means: {num_means}')
    axes[i].set_xlabel('Mean Value')
    axes[i].set_ylabel('Frequency')
    axes[i].legend(['Population Mean'])

plt.tight_layout()
plt.show()

In [ ]:
# 표본 데이터
data = [4.3, 4.1, 5.2, 4.9, 5.0, 4.5, 4.7, 4.8, 5.2, 4.6]

In [ ]:
# 파이썬으로 평균 계산(모분산을 모를 때)

from scipy.stats import t

mean = np.mean(data)
n = len(data)

se = np.std(data, ddof = 1) / np.sqrt(n)

# 95% 신뢰구간
print(round(mean - t.ppf(0.975, n-1) * se, 3), round(mean + t.ppf(0.975, n-1) * se, 3))
ci = t.interval(0.95, loc = mean, scale = se, df = n - 1)
print("95% 신뢰구간 : ", *[round(i, 3) for i in ci])

#### 통계적 검정

In [ ]:
# 귀무가설(H0) / 대립가설(H1)
# 미국 여행 만족도 조사 =>
#귀무가설 = '한국인의 미국여행 만족도의 평균이 80점이다'
# 30명을 뽑았을 때의 평균은 83점

from scipy.stats import norm

std = 5 / np.sqrt(30)
print(norm.sf(83, loc = 80, scale = std))
# 유의수준 5%(0.005)보다 작게 나왔으므로, 귀무가설 기각.

#### 검정통계량

In [ ]:
# Z 검정통계량 / 스튜던트 t 검정통계량 등이 있음.

### Section 02 학습 : t-검정과 분산 비교

#### t 검정

기본적으로 t-검정은 평균을 가지고 특정과 같은지 검정을 진행하는 방식임.   
귀무가설 기각의 기준은 p-value가 유의수준보다 작은지, 큰지임.   

p-value < 0.05 = 귀무가설(당연한사실) 기각   

1. 단일 표본(1 표본) 검정 (=One sample t-test)
    - 주어진 데이터의 평균이 특정 값과 같다(귀무가설)
2. 독립 2 표본 t 검정 (=Independent two-sample t-test)
    -  두 개의 독립된 집단의 평균이 동일하다(귀무가설)
3. 대응 표본 t-검정(=Paired sample t-test)
    - 동일한 대상에서 두 번의 측정을 통해 얻은 데이터의 평균 차이가 없다(귀무가설)

In [ ]:
# 단일표본 t 검정
# 귀무가설 = 학생들의 점수 평균은 10이다. 유의수준 5%
sample = [9.76, 11.1, 10.7, 10.72, 11.8, 6.15, 10.52, 14.83, 13.03, 16.46, 10.84, 12.45]

from scipy.stats import ttest_1samp

# print(ttest_1samp.__doc__)

t_statistic, p_value = ttest_1samp(sample, popmean = 10, alternative = 'two-sided')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)

# p-value가 0.05보다 크기 때문에 귀무가설 채택.

In [ ]:
# 독립 2 표본 t-검정
import pandas as pd

sample = [9.76, 11.1, 10.7, 10.72, 11.8, 6.15, 10.52, 14.83, 13.03, 16.46, 10.84, 12.45]
gender = ["Female"]*7 + ["Male"]*5

my_tab2 = pd.DataFrame({"score" : sample, "gender" : gender})
# print(my_tab2)

from scipy.stats import ttest_ind

male = my_tab2[my_tab2['gender'] == 'Male']
female = my_tab2[my_tab2['gender'] == 'Female']

t_statistic, p_value = ttest_ind(male['score'], female['score'], equal_var = True, alternative = 'greater')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)
# 유의수준 5%에서 p-value가 작기 때문에, 귀무가설 기각. 남성 평균이 여성보다 더 크다는 대립가설을 채택할 수 있음.
# 대립가설 관련 설정은 alternative = 'greater' 파라미터로 할 수 있음.

In [ ]:
t_statistic, p_value = ttest_ind(male['score'], female['score'], equal_var = True, alternative = 'greater')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)

t_statistic, p_value = ttest_ind(female['score'], male['score'], equal_var = True, alternative = 'less')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)

In [ ]:
# 대응 표본 t-검정

import numpy as np
before = np.array([9.76, 11.1, 10.7, 10.72, 11.8, 6.15])
after = np.array([10.52, 14.83, 13.03, 16.46, 10.84, 12.45])

from scipy.stats import ttest_rel

# print(ttest_rel.__doc__)

t_statistic, p_value = ttest_rel(after, before, alternative = 'greater')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)
# 유의수준보다 p value가 작으므로, 귀무가설 기각.

In [ ]:
# 대응 표본 t-검정은 단일 표본 t-검정으로도 계산할 수 있음.
from scipy.stats import ttest_1samp
sample_d = after - before

t_statistic, p_value = ttest_1samp(sample_d, popmean = 0, alternative = 'greater')
print("t_statistic: ", t_statistic)
print("p-value: ", p_value)
# 0.05 보다 작으므로, 귀무가설 기각. 대립 가설 after > before 채택.

In [ ]:
# 독립 2 표본 t-검정(분산 동일하지 않을 때)
# 남녀 score sample 활용.
result = ttest_ind(male['score'], female['score'], equal_var = False, alternative = 'greater')
print(result)

#### F-검정

 두 그룹의 분산이 같은지 판단하는 검정법.

In [ ]:
import numpy as np

oj_lengths = np.array([17.6, 9.7, 16.5, 12.0, 21.5, 23.3, 23.6, 26.4, 20.0, 25.2, 25.8, 21.2, 14.5, 27.3, 23.8]) # OJ 그룹 데이터 (15개)
vc_lengths = np.array([7.6, 4.2, 10.0, 11.5, 7.3, 5.8, 14.5, 10.6, 8.2, 9.4, 16.5, 9.7, 8.3, 13.6, 8.2]) # VC 그룹 데이터 (15개)

In [ ]:
s1 = oj_lengths.std(ddof = 1)
s2 = vc_lengths.std(ddof = 1)

ratio_of_variances = s1**2 / s2**2

print("ratio_of_variances: ", round(ratio_of_variances, 4))

In [ ]:
from scipy.stats import f

def f_test(x, y, alternative = "two_sided"):
    x = np.array(x)
    y = np.array(y)
    df1 = len(x) - 1
    df2 = len(y) - 1
    f_stat = np.var(x, ddof = 1) / np.var(y, ddof = 1)
    if alternative == 'greater':
        p = 1.0 - f.cdf(f_stat, df1, df2)
    elif alternative == 'less':
        p = f.cdf(f_stat, df1, df2)
    else:
        p = 1-f.cdf(f_stat, df1, df2)
        p = 2.0 * min(p, 1-p)
    return f_stat, p

f_value, p_value = f_test(oj_lengths, vc_lengths)

print("Test statistic: ", f_value)
print("p-value: ", p_value)
# 귀무가설 기각 X. 모분산이 동일함.

### Section 03 학습 : 데이터가 분포를 따르는지 확인하는 방법

#### 다섯숫자 요약(최솟값, 1사분위수, 중앙값, 3사분위수, 최댓값)

In [ ]:
import numpy as np
data = np.array([155, 126, 27, 82, 115, 140, 73, 92, 110, 134])

sorted_data = np.sort(data)

minimum = np.min(sorted_data)
maximum = np.max(sorted_data)

median = np.median(sorted_data)

lower = sorted_data[sorted_data < median]
upper = sorted_data[sorted_data > median]

q1 = np.median(lower)
q3 = np.median(upper)

print(minimum)
print(q1)
print(median)
print(q3)
print(maximum)
print("IQR", q3 - q1)

#### Quantile - Quantile plot

In [ ]:
data = np.array([155, 126, 27, 82, 115, 140, 73, 92, 110, 134])
sorted_data = np.sort(data)
n = len(data)

q3_percentile = (n-1) * 0.75 + 1
j3, h3 = divmod(q3_percentile, 1)
q3 = (1 - h3) * sorted_data[int(j3) - 1] + h3 * sorted_data[int(j3)]
print(q3)

# int(j3)-1 와 int(j3)는 0-based 인덱스로 각각 6(=7th)과 7(=8th)
# 값들: sorted_data[6] = 126, sorted_data[7] = 134
# 보간: q3 = 0.25*126 + 0.75*134 = 31.5 + 100.5 = 132.0

In [ ]:
# np.percentile 을 사용해서 백분위에 해당되는 백분위수 구하기.
import numpy as np

x = np.array([155, 126, 27, 82, 115, 140, 73, 92, 110, 134])
q1 = np.percentile(x, 25)
q2 = np.percentile(x, 50)
q3 = np.percentile(x, 75)

print(q1)
print(q2)
print(q3)

In [ ]:
# 데이터에 대응되는 백분위 구하기
import numpy as np
import scipy.stats as sp

x = np.array([155, 126, 27, 82, 115, 140, 73, 92, 110, 134])
percentiles = np.array([sp.percentileofscore(x, value, kind = 'rank') for value in x])
print(percentiles)

In [ ]:
# QQ plot 그리기
import numpy as np
import scipy.stats as sp

data_x = np.array([4.62, 4.09, 6.2, 8.24, 0.77, 5.55, 3.11, 11.97, 2.16, 3.24, 10.91, 11.36, 0.87])

percentile_rank = np.array([sp.percentileofscore(data_x, value, kind = 'rank') for value in data_x])
print(percentile_rank[:6])

In [ ]:
# 이론적으로 계산한 백분위수
theory_x = sp.norm.ppf(percentile_rank/100, np.mean(data_x), np.std(data_x))
print(theory_x[:6])

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(theory_x, data_x, color = 'k')
plt.plot([0,12], [0,12], 'k')
plt.title('QQplot')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.show()

In [ ]:
sp.probplot(data_x, dist = 'norm', plot = plt)

#### Shapiro-Wilk 검정
귀무가설 = 데이터가 정규분포를 따른다.

In [ ]:
from scipy.stats import shapiro

data_x = np.array([4.62, 4.09, 6.2, 8.24, 0.77, 5.55, 3.11, 11.97, 2.16, 3.24, 10.91, 11.36, 0.87])
w, p_value = shapiro(data_x)

print("W:", w)
print("p-value:", p_value)
# 귀무가설을 기각하지 못함.

#### 앤더슨-달링 검정
이론적인 누적분포함수와 데이터에서 뽑혀진 누적분포함수가 얼마나 비슷한지 체크하여 검정하는 방법.   
누적분포함수이기 때문에 검정통계량, p-value를 바로 계산할 수 없어서, 유의수준 체크를 할 때 해당 데이터에 대한 임계값과 검정통계량 수치를 보고 귀무가설 기각 여부를 정해야 함.

In [ ]:
# 주어진 데이터가 정규분포를 따르는 지, 앤더슨 달링 검정 진행해보기.
# 귀무가설 : 데이터가 정규분포를 따른다.
from scipy.stats import anderson, norm

sample_data = np.array([4.62, 4.09, 6.2, 8.24, 0.77, 5.55, 3.11, 11.97, 2.16, 3.24, 10.91, 11.36, 0.87])

result = anderson(sample_data, dist = 'norm')
print('검정통계량', result[0])
print('임계값', result[1])
print('유의수준', result[2])

# 유의수준 5%에 해당되는 임계값은 0.679 / 계산한 검정통계량릉 0.424임.
# 유의수준보다 검정통계량이 작기 때문에, 누적분포 간 차이가 적음. 귀무가설 채택.
# 기존의 다른 검정 방식과 다름!

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

ecdf = ECDF(sample_data)
x = np.linspace(min(sample_data), max(sample_data))
y = ecdf(x)

plt.scatter(x, y)
plt.title("Estimated CDF vs. CDF")

# add Normal CDF
k = np.arange(min(sample_data), max(sample_data), 0.1)
plt.plot(k, norm.cdf(k, loc = np.mean(sample_data), scale = np.std(sample_data, ddof = 1)), color = 'k')
plt.show( )

#### 카이제곱 검정
주어진 데이터가 기대하는 분포나 독립성, 동질성을 갖는지 검증할 때 활용.   
자주 나오는 문제임.

In [ ]:
# 1 표본 분산 검정
# 검정통계량(t) = (n-1) * 표본분산 / 모분산

# 분산이 1.3을 넘는 경우, 부적격이라 판단함. 귀무가설 = 분산 1.3을 넘지 않는다. 유의수준 5%

import numpy as np
from scipy.stats import chi2

sample_data = [10.67, 9.92, 9.62, 9.53, 9.14, 9.74, 8.45, 12.65, 11.47, 8.62]
n = len(sample_data)
sigma0_var = 1.3
df = n -1

sample_variance = np.var(sample_data, ddof = 1)

# 검정통계량
t = df * sample_variance / sigma0_var

p_value1 = 1 - chi2.cdf(t, df = df)
p_value2 = chi2.sf(t, df = df)

print(p_value1)
print(p_value2)

In [ ]:
# 독립성 검정
# 운동선수, 일반인에 대한 흡연 여부 조사 데이터이다. 운동선수와 흡연 여부 간의 독립성 검정을 수행하시오.

import numpy as np
from scipy.stats import chi2_contingency

table = np.array([[14, 4], [0, 10]])

chi2, p, df, expected = chi2_contingency(table, correction = False)
print(chi2.round(3), df, p.round(4))

# 귀무가설 기각. 운동선수와 흡연 여부는 독립적이지 않음.

In [ ]:
# 동질성 검정
# 두 개 이상의 모집단에서 추출된 표본들의 카테고리 분포 또는 비율이 동일한지 검정. 교차표 형태로 데이터가 제공됨.

import numpy as np
from scipy.stats import chi2_contingency

table = np.array([[50, 30, 20], [45, 35, 20]])
# chi2, p, df, expected = chi2_contingency(table, correction = False)
result = chi2_contingency(table, correction = False)

print(result)
# p value가 유의수준 5%에서 더 크기 때문에, 귀무가설을 기각하지 않음. 분포가 동질함.

In [ ]:
# 적합도 검정
# 데이터가 특정 이론적 분포를 따르는지.

import numpy as np
from scipy.stats import chisquare

observed = np.array([13, 23, 24, 20, 27, 18, 15])
# 기대빈도 리스트. 출생에 대한 기대빈도는 1/7이고, 140명에 대해서이기 때문에 기대빈도는 [20, 20, 20, 20, 20, 20, 20]임.
expected = np.repeat(20, 7)

statistic, p_value = chisquare(observed, f_exp = expected)
print(statistic.round(3))
print(p_value.round(4))
# p-value가 유의수준보다 크기 때문에, 귀무가설 기각하지 못함. 요일별 출생률은 같다.

### Section 04 학습 : 분산분석(ANOVA)
주로 3개 이상 집단에서 평균의 차이를 검정하기 위해 사용되는 방법.
- One way ANOVA : 한가지 관심변수에 대한 그룹 간 평균 차이.
- Two way ANOVA : 두 가지 관심변수에 대한 그룹 간 평균 차이.

In [ ]:
# One way ANOVA
import pandas as pd
import numpy as np

# 데이터 입력
scents = ['Lavender', 'Rosemary', 'Peppermint']
minutes_lavender = [10, 12, 11, 9, 8, 12, 11, 10, 10, 11]
minutes_rosemary = [14, 15, 13, 16, 14, 15, 14, 13, 14, 16]
minutes_peppermint = [18, 17, 18, 16, 17, 19, 18, 17, 18, 19]

anova_data = pd.DataFrame({
 'Scent' : np.repeat(scents, 10),
 'Minutes' : minutes_lavender + minutes_rosemary + minutes_peppermint
})
anova_data.head()

In [ ]:
anova_data.groupby(['Scent']).describe()

In [ ]:
from scipy.stats import f_oneway

# 각 그룹의 데이터를 추출
lavender = anova_data[anova_data['Scent'] == 'Lavender']['Minutes']
rosemary = anova_data[anova_data['Scent'] == 'Rosemary']['Minutes']
peppermint = anova_data[anova_data['Scent'] == 'Peppermint']['Minutes']

# 일원 분산분석(One-way ANOVA) 수행
f_statistic, p_value = f_oneway(lavender, rosemary, peppermint)
print(f'F-statistic: {f_statistic}, p-value: {p_value}')

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 모델 적합
model = ols('Minutes ~ C(Scent)', data = anova_data).fit()

# ANOVA 수행하기
anova_results = sm.stats.anova_lm(model, typ = 2)
print(anova_results)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(model.fittedvalues, model.resid)
plt.show()

In [ ]:
import scipy.stats as sp
sp.probplot(model.resid, dist = "norm", plot = plt)

In [ ]:
W, p = sp.shapiro(model.resid)
print(f'검정통계량: {W: .3f}, 유의확률: {p: .3f}')

In [ ]:
from scipy.stats import bartlett

groups = ['Lavender', 'Rosemary', 'Peppermint']
grouped_residuals = [model.resid[anova_data['Scent'] == group] for group in groups]

test_statistic, p_value = bartlett(*grouped_residuals)
print(f"검정통계량: {test_statistic}, p-value: {p_value}")

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Tukey HSD 사후 검정
tukey = pairwise_tukeyhsd(endog = anova_data['Minutes'], groups = anova_data['Scent'], alpha = 0.05)
print(tukey)

### Section 05 학습 : 비모수검정

In [ ]:
import numpy as np
from scipy.stats import rankdata
sample = np.array([9.76, 11.1, 10.7, 10.72, 11.8, 6.15, 10.52, 14.83, 13.03, 16.46, 10.84, 12.45])

sample_diff = abs(np.array(sample) - 10)
r_i = rankdata(sample_diff)
psi_i = np.where(sample - 10 >= 0, 1, 0)

sum(r_i[psi_i > 0])

In [ ]:
from scipy.stats import wilcoxon
eta_0 = 10
statistics, pvalue = wilcoxon(sample-eta_0, alternative = "two-sided")

print("Test statistic: ", statistics)
print("p-value: ", pvalue)

In [ ]:
a = np.arange(start = 1, stop = 13)
print(sum(a)-statistics)

In [ ]:
import pandas as pd
sample = [9.76, 11.1, 10.7, 10.72, 11.8, 6.15, 10.52, 14.83, 13.03, 16.46, 10.84, 12.45]
gender = ['female'] * 7 + ['male'] * 5
data_mww = pd.DataFrame({'id' : range(1,13), 'score' : sample, 'gender' : gender})

In [ ]:
from scipy.stats import rankdata
# 표본 크기 계산
n1 = len(data_mww[data_mww["gender"] == "female"])
n2 = len(data_mww) - n1

# 순위합 계산하기
r_i = rankdata(data_mww["score"])
r_1p = sum(r_i[ : 7])
r_2p = sum(r_i[7 : ])

# 검정통계량 계산
u1 = n1 * n2 + sum(range(1, n1 + 1)) - r_1p
u2 = n1 * n2 + sum(range(1, n2 + 1)) - r_2p
U = min(u1, u2)

print(U)

In [ ]:
from scipy.stats import mannwhitneyu
female = data_mww[data_mww['gender'] == 'female']['score']
male = data_mww[data_mww['gender'] == 'male']['score']

stat, pvalue = mannwhitneyu(female, male, method = "exact")

print("stat: ", stat.round(3))
print("p-value: ", pvalue.round(3))

In [ ]:
from scipy.stats.mstats import brunnermunzel

female = data_mww[data_mww['gender'] == 'female']['score']
male = data_mww[data_mww['gender'] == 'male']['score']

stat, pvalue = brunnermunzel(female, male, alternative = 'two-sided')

print("stat: ", stat.round(4))
print("p-value: ", pvalue.round(4))

In [ ]:
import pandas as pd
import numpy as np

id = [1, 2, 3, 4, 5, 6]
before_after = ['before'] * 6 + ['after'] * 6
tab3 = pd.DataFrame({'id': id * 2, 'score': sample, 'group': before_after})

# 자료 변환
test3_data = tab3.pivot(index = 'id', columns = 'group', values = 'score')
test3_data['score_diff'] = test3_data['after'] - test3_data['before']

print(test3_data['score_diff'])

In [ ]:
sample_sign = np.sign(test3_data['score_diff'])
sum(rankdata(abs(test3_data['score_diff']))[sample_sign > 0])

In [ ]:
from scipy.stats import wilcoxon
statistics, pvalue = wilcoxon(test3_data['score_diff'], alternative = 'greater')

print("Test statistic: ", statistics)
print("p-value: ", pvalue)

#### 비모수 분산검정

In [ ]:
import pandas as pd
import numpy as np
mydata=pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/tooth_growth.csv')
# 총 관찰 데이터는 30개
mydata.head()

In [ ]:
from scipy.stats import levene
a = mydata[mydata['supp'] == 'OJ']['len']
b = mydata[mydata['supp'] == 'VC']['len']

statistics, pvalue = levene(a, b, center = 'mean')
print("Test statistic: ", statistics)
print("p-value: ", pvalue)

#### 표본 부호 검정(Sign test)

In [ ]:
sample_sign = np.sign(np.array(sample) - 10)
sum(sample_sign > 0)

In [ ]:
from statsmodels.stats.descriptivestats import sign_test
sample = np.array([9.76, 11.1, 10.7, 10.72, 11.8, 6.15, 10.52, 14.83, 13.03, 16.46, 10.84, 12.45])
statistics, pvalue = sign_test(sample, mu0 = 10)
print("Test statistic: ", statistics)
print("p-value: ", pvalue)

In [ ]:
from scipy.stats import binom
(1 - binom.cdf(9, 12, 0.5)) * 2